In [89]:
import pandas as pd 
import numpy as np 
from itertools import product

In [101]:
countries = ['us','de','dk','fr','it','nl','se','sp']

In [102]:
test = pd.read_pickle('output/params_ref_us.pkl')
test

,value,free,se
name,,,
sigma,2.101034,True,0.109400
beta,0.970000,False,NaN
phi,0.282108,True,0.016309
psi,0.136652,True,0.012310
delta_h1,-1.021047,True,0.115155
delta_h2,3.430393,True,0.045294
eta,0.000000,False,NaN
tfp,1.000000,False,NaN
price,1.000000,False,NaN


In [103]:
pars_select = ['delta_h1','delta_h2','price','tfp']
stat = ['par','se']
tuples = list(product(*[pars_select,stat]))
table = pd.DataFrame(index=pd.MultiIndex.from_tuples(tuples),columns=countries)
for c in countries: 
	df = pd.read_pickle('output/params_ref_'+c+'.pkl')
	for p in pars_select:
		table.loc[(p,'par'),c] = df.loc[p,'value']
		table.loc[(p,'se'),c] = df.loc[p,'se']


us        de        dk        fr        it        nl  \
delta_h1 par -1.021047 -1.297368 -1.620886 -0.989299 -0.796356 -1.231644   
         se   0.115155  0.015218  0.038952  0.000937  0.037939  0.007234   
delta_h2 par  3.430393  4.111966  4.265841  3.769468  3.933223  3.955199   
         se   0.045294  0.019832  0.039728  0.042211  0.072573  0.076796   
price    par       1.0  0.835118  0.887434  0.603227  0.606923  0.648879   
         se        NaN  0.050692   0.02848  0.017388  0.050313  0.113703   
tfp      par       1.0  1.022368  1.259887  0.937066  0.676091  1.039062   
         se        NaN  0.039218  0.041073  0.025487  0.007936  0.073851   

                    se        sp  
delta_h1 par -1.419716  0.176934  
         se    0.07709  0.014202  
delta_h2 par  4.317404  3.519121  
         se   0.192694  0.116488  
price    par   0.88398  0.851795  
         se   0.079758  0.159627  
tfp      par  0.805936  0.837693  
         se   0.060558  0.071441

In [104]:
pars_select = ['delta_h1','delta_h2','price','tfp']
labels = ['$\\alpha_{10}$','$\\alpha_{11}$','$\\frac{p}{p_{US}}$','$\\frac{A}{A_{US}}$']
maps_labels = dict(zip(pars_select,labels))
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','se']])))

In [106]:
for c in table.columns:
	table[c] = table[c].astype('float64')
	table[c] = table[c].round(3)
	table[c] = table[c].astype('str')
	table[c] = np.where(table[c]=='nan','-',table[c])
	for s in table.index:
		if s[1]=='se':
			if table.loc[s,c]!='-':
				fmt = "({se})"
				table.loc[s,c] = fmt.format(se=table.loc[s,c])
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','']])))

In [107]:
table

us       de       dk       fr       it       nl  \
$\alpha_{10}$         -1.021   -1.297   -1.621   -0.989   -0.796   -1.232   
                     (0.115)  (0.015)  (0.039)  (0.001)  (0.038)  (0.007)   
$\alpha_{11}$           3.43    4.112    4.266    3.769    3.933    3.955   
                     (0.045)   (0.02)   (0.04)  (0.042)  (0.073)  (0.077)   
$\frac{p}{p_{US}}$       1.0    0.835    0.887    0.603    0.607    0.649   
                           -  (0.051)  (0.028)  (0.017)   (0.05)  (0.114)   
$\frac{A}{A_{US}}$       1.0    1.022     1.26    0.937    0.676    1.039   
                           -  (0.039)  (0.041)  (0.025)  (0.008)  (0.074)   

                          se       sp  
$\alpha_{10}$          -1.42    0.177  
                     (0.077)  (0.014)  
$\alpha_{11}$          4.317    3.519  
                     (0.193)  (0.116)  
$\frac{p}{p_{US}}$     0.884    0.852  
                      (0.08)   (0.16)  
$\frac{A}{A_{US}}$     0.806    0.838  
                     (0.061)  (0.071)

In [108]:
table.to_latex('../tables/table_8_country_specific_estimates.tex')

## Average US-EU differences

This number is used in counterfactual analysis. 

In [111]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{p}{p_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.7595714285714287

In [112]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{A}{A_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.9397142857142857